# Generative Adversarial Network
## GAN 介紹
生成對抗網路(Generative Adversarial Network, GAN)，GAN的架構是由Generator(生成器)和Discriminator(鑑別器)所組成。  
**Generator:** 負責生成圖片。  
**Discriminator:** 負責辨識圖片的真實性，真實圖片給予高分，生成圖片給予低分。  
如果生成圖片越接近真實圖片，Discriminator會給生成圖片越高的分數，Discriminator的輸出會經過Sigmoid激勵函數，所以分數會介於0 ~ 1 間。

<img src="img/gan_1.jpg" width="500" style="float:left">  

## GAN訓練及損失函數
GAN的訓練方式與以往不同，需將Generator和Discriminator分開訓練。但兩個是交互訓練的，例如先訓練Discriminator N次，在訓練Generator一次，而N為超參數。

<img src="img/gan_2.jpg" width="500" style="float:left">

### Generator training
目的是追求最低的Generator Loss。訓練時需將Discriminator的權重固定住，只訓練Generator的權重，Generator Loss越低，代表Discriminator認為Generator產生接近真實的圖片。Generator Loss 公式如下:  
<img src="img/gan_3.jpg" width="500" style="float:left">
<img src="img/gan_4.jpg" width="500" style="float:left">

### Discriminator training
目的是追求最低的Discriminator Loss。訓練時需將Generator的權重固定住，只訓練Discriminator的權重，Discriminator Loss越低，代表Discriminator越有能力辨別出真實圖片與生成圖片。Discriminator Loss 公式如下:  
<img src="img/gan_5.jpg" width="500" style="float:left">
<img src="img/gan_6.jpg" width="500" style="float:left">

對GAN來說，一個迭代(Iterative)的訓練，代表Generator和Discriminator至少都有訓練到一次。

# GAN、WGAN、WGAN-GP的演進
## GAN的問題
原始GAN的最大問題為訓練的不穩定性。原因如下:  
1. Discriminator訓練的太好，會導致Discriminator的輸出分布於"0"和"1"附近，也因為輸出層的激勵函數為Sigmoid，所以會有梯度消失的問題，導致Generator難以更新。  
2. Discriminator訓練的不好，會導致Discriminator對於真實圖片和生成圖片難以辨別，使兩者的輸出分佈重疊，最終Generator會有錯誤的判斷，然後Generator往錯誤的方向更新。  

基於上述的原因，Discriminator不易訓練到剛好的程度，需仔細觀察Discriminator Loss，才有機會讓Generator產生更真實的圖片。

在GAN的論文中提出Discriminator的最大化目標函數(Objective function)，定義如下:  
<img src="img/gan_7.jpg" width="800" style="float:left">

在固定的Generator下，可以得到最佳的Discriminator，定義如下:  
<img src="img/gan_8.jpg" width="300" style="float:left">

因此，Objective function可以從寫如下:  
<img src="img/gan_9.jpg" width="800" style="float:left">

其中:  
<img src="img/gan_10.jpg" width="500" style="float:left">

所以:  
<img src="img/gan_11.jpg" width="500" style="float:left">

最後:  
<img src="img/gan_12.jpg" width="500" style="float:left">

上式中的**KL(Kullback-Leibler Divergence, 又稱相對熵)**，是用來衡量兩個機率分佈的相似度，定義如下:  
<img src="img/gan_13.jpg" width="300" style="float:left">

**JSD(Jensen-Shannon Divergence)**，這也是用來衡量兩個機率分佈的相似度，定義如下:  
<img src="img/gan_14.jpg" width="300" style="float:left">

最佳的Discriminator公式最後是藉由JSD量測真實圖片和生成圖片間的相似度。當真實圖片和生成圖片分佈一致時，JSD為"0"，但在不相交時，JSD都是固定"log2"。
### 結論: 真實圖片和生成圖片分佈在不相交的情況下，Generator無法有效更新。

## Wasserstein distance 介紹
WGAN就是為了要解決一般GAN所遇到的問題。  
1. 首先WGAN將Discriminator的激勵函數Sigmoid移除，來避免梯度消失的問題。
2. 再來提出Wasserstein distance量測方法，來量測兩機率分佈間的距離。定義如下:  

<img src="img/gan_15.jpg" width="500" style="float:left">

Wasserstein distance可以用推土機來理解，Π包含所有可能土堆的運輸路徑，公式{E(x,y)~γ[||x - y||]}代表在γ這個運輸路徑下，將土堆Pr(真實)搬移到Pg(生成)所需的距離消耗，Wasserstein distance就是在路徑規劃(optimal transport plan)下的距離消耗。

Wasserstein distance可以根據Kantorovich-Rubinstein duality定理，轉換成如下:  
<img src="img/gan_16.jpg" width="500" style="float:left">

其中，1-Lipschitz主要是限制Discriminator，||D(x1) - D(x2)|| <= k ||x1 - x2||, k=1。  
Discriminator為一個複雜函數，並且滿足1-Lipschitz，即Discriminator函數的最大斜率不能超過1。  
如果不能滿足1-Lipschitz function的話，Discriminator會將Pg往-∞更新、Pr往+∞更新，模型訓練到最後就會發生崩潰。

為了滿足**1-Lipschitz function**，WGAN使用了**Weight clipping**的方式，限制Discriminator的權重大小，讓更新後的Discriminator權重必須在**超參數c**的控制範圍內(將權重大小限制在[-c, c]中)。公式如下:  
<img src="img/gan_17.jpg" width="300" style="float:left">

w: Discriminator權重  
c: 限制權重大小的超參數  

但使用Weight clipping的方法，使Discriminator強制滿足1-Lipschitz function，會遇到以下問題:  
1. WGAN使用Weight clipping方法訓練在Swiss Roll dataset上，並限制Discriminator權重的極大和極小值為[-0.01, 0.01]，而經過訓練後發現，Discriminator的大多數權重會集中在極大和極小值附近，這樣導致了模型無法描述複雜的問題。  
2. WGAN使用Weight clipping方法訓練在Swiss Roll dataset上。如果超參數c測定稍為大一點，每經過一層網路層，梯度就會變大一點，在經過多層傳遞後，就會發生梯度爆炸的問題；反之，如果超參數c測定稍為小一點，每經過一層網路層，梯度就會變小一點，在經過多層傳遞後，就會發生梯度消失的問題。

<img src="img/gan_18.jpg" width="600" style="float:left">  

因此，WGAN-GP提出Gradient Penalty的方法來解決上述的問題，同時也能滿足1-Lipschitz function。使用Gradient Penalty的方法訓練模型，Discriminator的權重分具多樣性，可以讓模型能夠描述更複雜的問題。詳細圖如上(c)。

WGAN-GP使用的Gradient Penalty是透過對梯度大小的限制來讓Discriminator滿足1-Lipschitz function。它的作法是在Pr(真實)和Pg(生成)抽樣後，將兩個樣本進行線性內插，並將數值帶入Discriminator計算梯度，而梯度值的歐幾里德範數(Euclidean norm)必須是越接近"1"越好，公式如下:  
<img src="img/gan_19.jpg" width="600" style="float:left">  

P(penalty): Pr(真實)和Pg(生成)抽樣後，將兩個樣本進行線性內插。

## WGAN-GP 損失函數
WGAN-GP與GAN的訓練流程相同，它將GAN架構的Discriminator最後一層的激勵函數Sigmoid移除，並換成線性輸出的，同時對Discriminator Loss和Grenerator Loss都做了改良，損失函數分別如下:  
1. Generator training目的是追求最低的Generator Loss，訓練時需將Discriminator的權重固定住，只訓練Generator的權重，Generator Loss越低，代表Discriminator認為Generator產生接近真實的圖片。

<img src="img/gan_20.jpg" width="400" style="float:left">

* N: 訓練資料量  
* D: Discriminator判斷器  
* G: Generator生成器  
* z: 從一個分佈中抽樣作為Generator的輸入

2. Discriminator training目的是追求最低的Discriminator Loss，訓練時需將Generator的權重固定住，只訓練Discriminator的權重，Discriminator Loss越低，代表Discriminator越有能力辨別出真實圖片與生成圖片。

<img src="img/gan_21.jpg" width="600" style="float:left">

* N: 訓練資料量  
* D: Discriminator判斷器  
* x: 從資料集中抽樣真實圖片  
* x': Generator生成的圖片  
* λ: Gradient Penalty係數，通常設為10

3. Gradient Penalty透過對梯度大小的限制來讓Discriminator滿足1-Lipschitz function。它的作法是在Pr(真實)和Pg(生成)抽樣後，將兩個樣本進行線性內插，並將數值帶入Discriminator計算梯度，而梯度值的歐幾里德範數(Euclidean norm)必須是越接近"1"越好。

<img src="img/gan_22.jpg" width="400" style="float:left">

* N: 訓練資料量  
* D: Discriminator判斷器  
* x~:是從真實圖片(x)和生成圖片(x')中抽樣，將兩個圖片樣本進行線性內插。  
* t: 為0到1之間均勻分佈的隨機抽樣數值。